Visto che i passi mi serviranno praticamente sempre in forma normalizzata, mi salvo anche la versione del dataset con la colonna dei passi normalizzati.
Inoltre probabilmente è meglio avere anche lo score in forma normalizzata.
Normalizzo anche le date usando minmax.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/valutati.csv.gz')

Normalizzo anche la data per poterla usare come feature.

In [3]:
from sklearn.preprocessing import MinMaxScaler
import datetime

df["date"]=pd.to_datetime(df["date"],format= "%Y-%m-%d")
df["n_date"] = MinMaxScaler().fit_transform(df[["date"]])


scale normalizza i dati separatamente per ogni utente, scale2 li normalizza tutti insieme. Scale2 è migliore in quanto mantiene un po' più informazione relativa a cosa è "generalmente normale"

In [4]:
from sklearn.preprocessing import StandardScaler  # Questo scaler è meno sensibile agli outliers


def scale(x):
    x["n_steps"] = StandardScaler().fit_transform(x[["steps"]])  # Lo scaling è fatto sulla base dell'utente
    x["n_sleep_score"] = StandardScaler().fit_transform(x[["sleep_score"]])  # Lo scaling è fatto sulla base dell'utente
    return x


def scale2(x):
    x["n_steps_all"] = StandardScaler().fit_transform(x[["steps"]])  # Lo scaling è fatto sulla base dell'utente
    x["n_sleep_score_all"] = StandardScaler().fit_transform(
        x[["sleep_score"]])  # Lo scaling è fatto sulla base dell'utente
    return x


df = df.transform(scale2)
df.head()

Qui invece si calcolano le medie di alcuni valori, basate tutte su una sliding window che copre tra 6 e 10 giorni consecutivi. Lavorare con la media permette di avere dati meno rumorosi.
Oltre alle medie con sliding window, calcolo anche i delta tra una media e la successiva.

In [6]:
def rolling_mean(x):
    y=x[::-1].rolling(window='10D', min_periods=6, on="date")[[
        "steps",
        # "n_steps",
        "n_steps_all",
        "sleep_score",
        # "n_sleep_score",
        "n_sleep_score_all"
    ]].mean()[::-1]
    x["m_steps"] = y["steps"]
    x["dm_steps"] = y["steps"].diff(1)
    # x["mn_steps"]= y["n_steps"]
    # x["dmn_steps"]= y["n_steps"].diff(1)
    x["mn_steps_all"]= y["n_steps_all"]
    x["dmn_steps_all"]= y["n_steps_all"].diff(1)
    x["m_sleep_score"]= y["sleep_score"]
    x["dm_sleep_score"]= y["sleep_score"].diff(1)
    # x["mn_sleep_score"]= y["n_sleep_score"]
    # x["dmn_sleep_score"]= y["n_sleep_score"].diff(1)
    x["mn_sleep_score_all"] = y["n_sleep_score_all"]
    x["dmn_sleep_score_all"] = y["n_sleep_score_all"].diff(1)
    return x

df = df.groupby("uuid").apply(rolling_mean, include_groups=False)
df = df.rename_axis(["uuid", "id"]).reset_index().drop(columns=["id"])  # Per un csv più pulito, infatti il groupby aggiunge una colonna che non voglio
df.head(100)

In [7]:
df.to_csv('../data/scalati.csv.gz', compression='gzip', index=False)